<a href="https://www.kaggle.com/code/hntrnnggia/segnet-cnn?scriptVersionId=229358504" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
# Khởi tạo ImageDataGenerator cho train (có augmentation)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Chỉ cần rescale cho validation và test
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load tập train
train_generator = train_datagen.flow_from_directory(
    "/kaggle/input/btxrd-v1/btxrd_v1/train",
    target_size=(224, 294),
    batch_size=32,
    class_mode="categorical"
)

# Load tập validation
val_generator = val_test_datagen.flow_from_directory(
    "/kaggle/input/btxrd-v1/btxrd_v1/val",
    target_size=(224, 294),
    batch_size=32,
    class_mode="categorical"
)

# Load tập test
test_generator = val_test_datagen.flow_from_directory(
    "/kaggle/input/btxrd-v1/btxrd_v1/test",
    target_size=(224, 294),
    batch_size=32,
    class_mode="categorical",
    shuffle=False  # Đảm bảo thứ tự không thay đổi khi đánh giá
)

print(f"✅ Train samples: {train_generator.samples}")
print(f"✅ Validation samples: {val_generator.samples}")
print(f"✅ Test samples: {test_generator.samples}")
print("✅ Load dữ liệu hoàn tất!"

Found 2622 images belonging to 2 classes.
Found 375 images belonging to 2 classes.
Found 749 images belonging to 2 classes.
✅ Train samples: 2622
✅ Validation samples: 375
✅ Test samples: 749
✅ Load dữ liệu hoàn tất!


In [ ]:
# 3️⃣ Xây dựng mô hình SegNet
def build_segnet(input_shape=(224, 294, 3), num_classes=1):
    inputs = tf.keras.Input(shape=input_shape)

    # Encoder
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Decoder
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.UpSampling2D((2, 2))(x)

    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.UpSampling2D((2, 2))(x)

    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.UpSampling2D((2, 2))(x)

    # Output layer (Mask vùng ung thư)
    outputs = layers.Conv2D(num_classes, (1, 1), activation='sigmoid', padding='same')(x)

    model = models.Model(inputs, outputs, name="SegNet")
    return model

# 4️⃣ Huấn luyện SegNet
segnet_model = build_segnet()
segnet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

segnet_model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    batch_size=8,
    epochs=20
)

# 5️⃣ Lưu mô hình đã huấn luyện
segnet_model.save("segnet_model.h5")
print("✅ Mô hình SegNet đã được lưu!")